In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/nba-shot-logs/shot_logs.csv')
df.head(10)

In [ ]:
df = df.drop(['FGM','PTS','LOCATION','GAME_ID','MATCHUP','W','FINAL_MARGIN','GAME_CLOCK','CLOSEST_DEFENDER','CLOSEST_DEFENDER_PLAYER_ID','player_id','player_name'], axis=1)

df["SHOT_RESULT"][df.SHOT_RESULT == 'made'] = 1
df["SHOT_RESULT"][df.SHOT_RESULT == 'missed'] = 0

In [ ]:
df.head(10)

In [ ]:
allvar = df.columns.values.tolist()
allvar

In [ ]:
allvar.pop(7)
allvar

In [ ]:
X = df[allvar]
X.head()
# X = df[['SHOT_NUMBER', 'PERIOD', .... 'CLOSE_DEF_DIST']]

In [ ]:
y = df['SHOT_RESULT']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2,
                                                    random_state = 33)

In [ ]:
# dont look at this
y_test = y_test.astype('int32')
y_test.dtype

In [ ]:
from sklearn.impute import SimpleImputer

myimp = SimpleImputer()

#data = pd.DataFrame(myimp.fit_transform(data))

imputed_X_train = pd.DataFrame(myimp.fit_transform(X_train))
imputed_X_test = pd.DataFrame(myimp.transform(X_test))

imputed_X_train.columns = X_train.columns
imputed_X_test.columns = X_test.columns

X_train = imputed_X_train
X_test = imputed_X_test

In [ ]:
X_train.head()

In [ ]:
X_train['SHOT_CLOCK'] = round(X_train['SHOT_CLOCK'], 2)
X_test['SHOT_CLOCK'] = round(X_test['SHOT_CLOCK'], 2)

In [ ]:
X_train.head()

In [ ]:
import xgboost as xgb

# 转换格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Booster 超参数
hyperparams = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',  # 多分类的问题
    'num_class': 10,               # 类别数，与 multisoftmax 并用
    'gamma': 0.1,                  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 12,               # 构建树的深度，越大越容易过拟合
    'lambda': 2,                   # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,              # 随机采样训练样本
    'colsample_bytree': 0.7,       # 生成树时进行的列采样
    'min_child_weight': 3,
    'silent': 1,                   # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.007,                  # 如同学习率
    'seed': 1000,
    'nthread': 4,                  # cpu 线程数
    'eval_metric':'auc'
}

# mae, mse, rmse, auc, recall, precision

plst = list(hyperparams.items())
# evallist = [(dtest, 'eval'), (dtrain, 'train')]   # 指定验证集

In [ ]:
num_round = 10

bst = xgb.train(plst, dtrain, num_round) # train model

In [ ]:
y_pred = bst.predict(dtest)

In [ ]:
y_pred

In [ ]:
y_pred = y_pred.astype('int32')
y_pred.dtype

In [ ]:
y_test = y_test.astype('int32')
y_test.dtype
y_test

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred)
print("accuarcy: %.2f%%" % (accuracy*100.0))

In [ ]:
pd.DataFrame(y_test).head(10)

In [ ]:
pd.DataFrame(y_pred).head(10)

In [ ]:
xgb.plot_importance(bst)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X_train, y_train)
print(rf0.oob_score_)

In [ ]:
y_predprob = rf0.predict_proba(X)[:, 1]
print(y_predprob)
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

In [ ]:
y_train.isnull().sum()

In [ ]:
from sklearn.linear_model import LogisticRegression


LR = LogisticRegression()
LR.fit(X_train, y_train)
predict_results=LR.predict(feature_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
 
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                          scoring='neg_mean_squared_error')
 
grid_search.fit(X_train, y_train)

In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

iris = datasets.load_iris()
X,y = iris.data,iris.target
col = iris.target_names 

train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.3, random_state=1)   # 分训练集和验证集

hyperparameters = {
              'max_depth': [5, 10, 15, 20, 25],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'n_estimators': [500, 1000, 2000, 3000, 5000],
              'min_child_weight': [0, 2, 5, 10, 20],
}

xlf = xgb.XGBClassifier(max_depth=15,
            learning_rate=0.01,
            n_estimators=2000,
            min_child_weight=2)


gs = GridSearchCV(xlf, param_grid=hyperparameters, scoring='accuracy', cv=3)
gs.fit(train_x, train_y)

xlf.fit(train_x, train_y)

#print("Best score: %0.3f" % gs.best_score_)
#print("Best parameters set: %s" % gs.best_params_ )

In [ ]:
xgb.plot_importance(xlf)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(train_x, train_y)
print(rf0.oob_score_)

In [ ]:
from sklearn.linear_model import LogisticRegression


LR = LogisticRegression()
LR.fit(train_x, train_y)